<a href="https://colab.research.google.com/github/jihun0423/electric_predict/blob/main/electric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir('./gdrive/MyDrive/electric')

## 전력 데이터 전처리

In [3]:
import pandas as pd
df = pd.read_csv('train.csv')

In [4]:
df.head(5)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [5]:
df['날짜']=df['일시'].str.split(' ').str[0]
df['hour']=df['일시'].str.split(' ').str[1].astype('int')

In [6]:
df['날짜'] = pd.to_datetime(df.날짜)

In [7]:
df.head(5)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),날짜,hour
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,2022-06-01,0
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,2022-06-01,1
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,2022-06-01,2
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,2022-06-01,3
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,2022-06-01,4


### feature 생성

In [8]:
import numpy as np
df['hour'] = np.sin(2*np.pi*df['hour']/24)


In [9]:
df['dow'] = df['날짜'].dt.weekday
df['month'] = df['날짜'].dt.month
df['day'] = df['날짜'].dt.day
df['holiday'] = df['dow'].apply(lambda x: 0 if x<5 else 1)

In [11]:
from datetime import datetime
holiday = [datetime(2022, 6, 1), datetime(2022, 6, 6), datetime(2022, 8, 15)]
df['holiday']=((df['dow']>4) | (df['날짜'].isin(holiday))) *1

In [ ]:
# from datetime import datetime
# holiday = [datetime(2022, 6, 1), datetime(2022, 6, 6), datetime(2022, 8, 15)]
# df['holiday'] = df.apply(lambda x: 1 if ((df['날짜'].isin(holiday)) | (df['dow'] > 4)) else 0, axis=1)

In [12]:
df.head(4)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),날짜,hour,dow,month,day,holiday
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,2022-06-01,0.000000,2,6,1,1
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,2022-06-01,0.258819,2,6,1,1
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,2022-06-01,0.500000,2,6,1,1
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,2022-06-01,0.707107,2,6,1,1


In [ ]:
df['THI'] = 9/5*df['기온(C)'] - 0.55*(1-df['습도(%)']/100)*(9/5*df['습도(%)']-26)+32
df['THI'] = pd.cut(df['THI'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4])
df['THI'] = df['THI'].astype(int)

In [ ]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
for num in range(1,101,1):
    temp = df[df['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
df['CDH'] = cdhs

### 결측치 처리

In [13]:
df.isnull().sum()

num_date_time         0
건물번호                  0
일시                    0
기온(C)                 0
강수량(mm)          160069
풍속(m/s)              19
습도(%)                 9
일조(hr)            75182
일사(MJ/m2)         87913
전력소비량(kWh)            0
날짜                    0
hour                  0
dow                   0
month                 0
day                   0
holiday               0
dtype: int64

In [15]:
df['강수량(mm)']=df['강수량(mm)'].fillna(0)

In [16]:
df.groupby(['건물번호','month','hour'],as_index=False).mean()

<ipython-input-16-2a8318cf48cc>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['건물번호','month','hour'],as_index=False).mean()


,건물번호,month,hour,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),dow,day,holiday
0,1,6,-1.000000,25.610000,0.723333,2.906667,65.633333,0.340000,0.651667,2595.296,2.966667,15.5,0.333333
1,1,6,-0.965926,24.886667,1.133333,3.083333,67.433333,0.323333,0.307333,2419.520,2.966667,15.5,0.333333
2,1,6,-0.965926,25.966667,0.493333,2.960000,64.700000,0.286667,0.928000,2838.504,2.966667,15.5,0.333333
3,1,6,-0.866025,23.923333,0.756667,3.026667,70.633333,0.073333,0.061667,2230.592,2.966667,15.5,0.333333
4,1,6,-0.866025,26.123333,0.326667,3.143333,64.700000,0.280000,1.131000,3422.256,2.966667,15.5,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,100,8,0.707107,26.425000,1.208333,1.341667,88.125000,0.195833,NaN,1121.330,2.750000,12.5,0.291667
6596,100,8,0.866025,24.762500,0.979167,1.029167,95.833333,NaN,NaN,406.050,2.750000,12.5,0.291667
6597,100,8,0.866025,25.570833,2.050000,1.170833,92.250000,0.120833,NaN,980.010,2.750000,12.5,0.291667
6598,100,8,0.965926,24.864583,1.177083,1.014583,95.750000,0.029167,NaN,646.080,2.750000,12.5,0.291667


In [17]:
df_nummonthhour_mean = df.groupby(['건물번호','month','hour'],as_index=False).mean()

<ipython-input-17-dd149735d4ae>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_nummonthhour_mean = df.groupby(['건물번호','month','hour'],as_index=False).mean()


In [18]:
null_feature = ['풍속(m/s)','습도(%)','일조(hr)','일사(MJ/m2)']

In [48]:
for col in null_feature:
  for i in df[df[col].isnull()].index:
    num = df.loc[i,'건물번호']
    month = df.loc[i,'month']
    hour = df.loc[i,'hour']
    data_mean = df_nummonthhour_mean[(df_nummonthhour_mean['건물번호']==num)&(df_nummonthhour_mean['month']==month)&(df_nummonthhour_mean['hour']==hour)]
    df.loc[i,col] = data_mean[col].values

In [50]:
df['일조(hr)'] = df['일조(hr)'].fillna(0)

In [51]:
df['일사(MJ/m2)'] = df['일사(MJ/m2)'].fillna(0)

In [52]:
df.isnull().sum()

num_date_time    0
건물번호             0
일시               0
기온(C)            0
강수량(mm)          0
풍속(m/s)          0
습도(%)            0
일조(hr)           0
일사(MJ/m2)        0
전력소비량(kWh)       0
날짜               0
hour             0
dow              0
month            0
day              0
holiday          0
dtype: int64

In [65]:
df

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),날짜,hour,dow,month,day,holiday
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,0.00,0.000000,1085.28,2022-06-01,0.000000,2,6,1,1
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,0.00,0.000000,1047.36,2022-06-01,0.258819,2,6,1,1
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,0.42,1.396667,974.88,2022-06-01,0.500000,2,6,1,1
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,0.00,0.000000,953.76,2022-06-01,0.707107,2,6,1,1
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,0.00,0.000000,986.40,2022-06-01,0.866025,2,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,0.0,0.9,86.0,0.50,0.000000,881.04,2022-08-24,-0.965926,2,8,24,0
203996,100_20220824 20,100,20220824 20,22.4,0.0,1.3,86.0,0.00,0.000000,798.96,2022-08-24,-0.866025,2,8,24,0
203997,100_20220824 21,100,20220824 21,21.3,0.0,1.0,92.0,0.00,0.000000,825.12,2022-08-24,-0.707107,2,8,24,0
203998,100_20220824 22,100,20220824 22,21.0,0.0,0.3,94.0,0.00,0.000000,640.08,2022-08-24,-0.500000,2,8,24,0


In [ ]:
df_numdowhour_mean = df.groupby('건물번호','dow','hour',as_index=False).mean()['일시','전력소비량(kWh)'].rename({'전력소비량(kWh): kWh_ndh_mean'})
df_numdowhour_sd = df.groupby('건물번호','dow','hour',as_index=False).std()['일시','전력소비량(kWh)'].rename({'전력소비량(kWh): kWh_ndh_std'})
df_numdow_mean = df.groupby('건물번호','dow',as_index=False).mean()['일시','전력소비량(kWh)'].rename({'전력소비량(kWh): kWh_nh_mean'})
df_numdow_sd = df.groupby('건물번호','dow',as_index=False).std()['일시','전력소비량(kWh)'].rename({'전력소비량(kWh): kWh_nh_std'})

In [ ]:
df.merge(df_numdowhour_mean,on='일시',how='left')

## 건물 정보 전처리

In [54]:
df_b = pd.read_csv('building_info.csv')

In [55]:
df_b.head(5)

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000


In [61]:
df_b.isnull().sum()

건물번호            0
건물유형            0
연면적(m2)         0
냉방면적(m2)        0
태양광용량(kW)       0
ESS저장용량(kWh)    0
PCS용량(kW)       0
dtype: int64

In [60]:
df_b=df_b.replace('-',0)

In [64]:
df_b.groupby('건물유형',as_index=False).mean()

<ipython-input-64-127e8baf7384>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_b.groupby('건물유형',as_index=False).mean()


,건물유형,건물번호,연면적(m2),냉방면적(m2)
0,건물기타,8.0,1.084411e+05,5.417702e+04
1,공공,19.5,1.687602e+06,1.411875e+06
2,대학교,27.5,3.699181e+05,2.478968e+05
3,데이터센터,34.0,1.851530e+04,1.187450e+04
4,백화점및아울렛,40.5,1.036846e+05,5.771201e+04
5,병원,48.5,1.019759e+05,8.230490e+04
6,상용,56.5,9.267848e+04,6.076371e+04
7,아파트,64.5,2.615197e+05,1.418271e+05
8,연구소,72.5,1.118182e+05,7.612745e+04
9,지식산업센터,80.5,1.735248e+05,7.056204e+04


In [ ]:
conv_dict = dict(zip(list(df_b['건물유형'].unique()), [0,1,2,3,4,5,6,7,8,9,10,11]))
conv_dict

In [ ]:
df_b['건물유형'] = df_b['건물유형'].map(conv_dict)

In [ ]:
pip install -U pandasql

In [ ]:
from pandasql import sqldf

query = """select 풍속(m/s) from df group by 건물번호, month, 시간 """

df1 = sqldf(query)
